# Chapter 8. Data Wrangling: Join, Combine, and Reshape

*   8.1 Hierarchical Indexing

*   8.2 Combining and Merging Datasets

*   8.3 Reshaping and Pivoting

source: [McKinney, Wes. Python for Data Analysis. O'Reilly Media. 3rd Edition](https://wesmckinney.com/book/data-wrangling)

In [1]:
import numpy as np
import pandas as pd

## 8.1 Hierarchical Idexing

*Hierarchical indexing* is an important feature of pandas that enables us to have multiple (two or more) index levels on an axis.

In [2]:
data01 = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data01

a  1    0.434564
   2    0.496692
   3    0.776574
b  1    0.398667
   3    0.772776
c  1    0.203884
   2    0.980136
d  2    0.212731
   3    0.149499
dtype: float64

> The “gaps” in the index display mean “use the label directly above”:

In [3]:
# The multi-index of the dataset
data01.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

**Partial indexing**

In [4]:
# Selecting consecutive indices in the first index level
data01["b":"d"]

b  1    0.398667
   3    0.772776
c  1    0.203884
   2    0.980136
d  2    0.212731
   3    0.149499
dtype: float64

In [5]:
# Selecting non-consecutive indices in the first index level
data01.loc[["a", "d"]]

a  1    0.434564
   2    0.496692
   3    0.776574
d  2    0.212731
   3    0.149499
dtype: float64

In [6]:
# Selecting all of the values having the index value 2 from the second index level:

data01.loc[:, 2]

a    0.496692
c    0.980136
d    0.212731
dtype: float64

**Reshaping the dataset with .unstack() and .stack()**

In [7]:
data02 = data01.unstack()
data02

,1,2,3
a,0.434564,0.496692,0.776574
b,0.398667,NaN,0.772776
c,0.203884,0.980136,NaN
d,NaN,0.212731,0.149499


In [8]:
data02.stack()

a  1    0.434564
   2    0.496692
   3    0.776574
b  1    0.398667
   3    0.772776
c  1    0.203884
   2    0.980136
d  2    0.212731
   3    0.149499
dtype: float64

**Hierarchical indexing in `pandas` dataframe**

In [9]:
frame01 = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame01

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

**Hierarchical levels can have names**

In [10]:
frame01.index.names = ["key1", "key2"]
frame01.columns.names = ["state", "color"]
frame01

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

> `state` and `color` are not part of the row labels.

In [11]:
frame01.index.nlevels

2

In [12]:
frame01.columns.nlevels

2

In [13]:
frame01["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 8.1.1 Reording and Sorting Levels

**Swapping index levels**

In [14]:
frame01.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [15]:
# Sorting the 2nd index level
frame01.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [16]:
frame01.swaplevel(0, 1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [17]:
frame01.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

###  8.1.2 Summary Statistics by Level

In [18]:
frame01

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [19]:
frame01.groupby(level="key1").sum()

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

In [20]:
frame01.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [21]:
frame01

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [22]:
frame01.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 Indexing with a DataFrame’s columns 

In [23]:
frame02 = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame02

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [24]:
frame03 = frame02.set_index(["c", "d"])
frame03

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [25]:
frame02.set_index(["c", "d"], drop=True)

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [26]:
frame02.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [27]:
frame03.reset_index().sort_index(axis=1, inplace=True)

In [28]:
frame02

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


## 8.2 Combining and Merging Datasets

Data contained in pandas objects can be combined in a number of ways:

*  `pandas.merge`

   *   Connect rows in DataFrames based on one or more keys. 

*  `pandas.concat`

   *   Concatenate or "stack" objects together along an axis.

*  `combine_first`

   *   Splice together overlapping data to fill in missing values in one object with values from another.

###  8.2.1 Database-Style DataFrame Joins

*Merge* or *join* operations combine datasets by linking rows using one or more keys. 

By default, `pandas.merge` does an "inner" join.
*   the keys in the result are the intersection, or the common set found in both tables. 

In [29]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [30]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


**many-to-one join**

In [31]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


> It's a good practice to specify the column to join explicitly.

In [32]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


**The column names are different in each object, Specify them separately.**

In [33]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [34]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [35]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [36]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


By default, `pandas.merge` does an "inner" join.
*   the keys in the result are the intersection, or the common set found in both tables. 

In [43]:
# how="inner": Use only the key combinations observed in both tables
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how = "inner")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [47]:
# how="left": Use all key combinations found in the left table
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how = "left")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,c,3,NaN,<NA>
4,a,4,a,0
5,a,5,a,0
6,b,6,b,1


In [48]:
# how="right": Use all key combinations found in the right table
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how = "right")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,NaN,<NA>,d,2


The outer join takes the union of the keys.
*   combining the effect of applying both left and right joins.

In [49]:
# how="outer": Use all key combinations observed in both tables together
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how = "outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


**On Merging DataFrames with than One Key**

In [37]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})

In [38]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [39]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [41]:
# how="inner": Use only the key combinations observed in both tables
pd.merge(left, right, on=["key1","key2"], how = "inner")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


In [42]:
# how="left": Use all key combinations found in the left table
pd.merge(left, right, on=["key1", "key2"], how="left")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6


In [50]:
# how="right": Use all key combinations found in the right table
pd.merge(left, right, on=["key1", "key2"], how="right")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6
3,bar,two,<NA>,7


In [ ]:
# how="outer": Use all key combinations observed in both tables together
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


In [40]:
# how="outer": Use all key combinations observed in both tables together
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


> Merging dataframes with overlapping column names

In [53]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [52]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 8.2.2 Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index (row labels).

In [105]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [109]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
right1

,group_val
a,3.5
b,7.0


In [62]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [63]:
pd.merge(left1, right1, left_on="key", right_index=True, how = "inner")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [64]:
pd.merge(left1, right1, left_on="key", right_index=True, how = "outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


**Merging on hierarchically indexed data**

In [66]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [68]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth_index

MultiIndex([('Nevada', 2001),
            ('Nevada', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2001),
            (  'Ohio', 2002)],
           )

In [70]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [71]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [72]:
# default: "how=inner"
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [73]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


**Using the indexes of both sides of the merge is also possible.**

In [74]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")

In [75]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [76]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [82]:
pd.merge(left2, right2, how="inner", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3,4,9,10
e,5,6,13,14


In [83]:
pd.merge(left2, right2, how="left", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
c,3,4,9,10
e,5,6,13,14


In [84]:
pd.merge(left2, right2, how="right", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [85]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


**DataFrame has a join instance method to simplify merging by index.**

In [87]:
left2.join(right2, how="inner")

,Ohio,Nevada,Missouri,Alabama
c,3,4,9,10
e,5,6,13,14


In [86]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


**Compared with `pandas.merge`, `DataFrame`’s join method performs a left join on the join keys by default.**

**It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame**

In [89]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [110]:
right1

,group_val
a,3.5
b,7.0


In [111]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [98]:
left1.join(right1, on="key", how="left").sort_index()

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [95]:
left1.join(right1, on="key", how="outer").sort_index()

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [124]:
left1.join(right1, on="key", how="right").sort_index()

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [125]:
left1.join(right1, on="key", how="inner").sort_index()

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [121]:
right1b = right1.copy(deep=True)
right1b.loc["d"] = [3]
right1b

,group_val
a,3.5
b,7.0
d,3.0


In [126]:
left1.join(right1b, on="key", how="left").sort_index()

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [127]:
left1.join(right1b, on="key", how="outer").sort_index()

,key,value,group_val
0.0,a,0,3.5
1.0,b,1,7.0
2.0,a,2,3.5
3.0,a,3,3.5
4.0,b,4,7.0
5.0,c,5,NaN
NaN,d,<NA>,3.0


### 8.2.3 Concatenating Along an Axis

**Reading assignment**

### 8.2.4 Combining Data with Overlap
**Reading assignment**

## 8.3 Reshaping and Pivoting

### 8.3.1  Reshaping with Hierarchical Indexing

In [129]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


**`.stack()`  “rotates” or pivots from the columns in the data to the rows.**

In [130]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

**`.unstack()` pivots from the rows into the columns.**

In [131]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [135]:
result.unstack(level=1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [133]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [134]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


**Unstacking might introduce missing data if all of the values in the level aren’t found in each subgroup.**

In [136]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [137]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [138]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [139]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

**When you unstack in a DataFrame, the level unstacked becomes the lowest level in the result.**

In [141]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [142]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [ ]:
**As with unstack, when calling stack we can indicate the name of the axis to stack.**

In [146]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [148]:
df.unstack()

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [150]:
df.unstack().stack(level="number")

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [152]:
df.unstack().stack(level="side")

number          one  three  two
state    side                  
Ohio     left     0      2    1
         right    5      7    6
Colorado left     3      5    4
         right    8     10    9

In [157]:
df.unstack(level="number").stack(level="number")

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [159]:
df.unstack(level="number").stack(level="side")

number          one  three  two
state    side                  
Ohio     left     0      2    1
         right    5      7    6
Colorado left     3      5    4
         right    8     10    9

In [161]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [162]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [165]:
df.unstack(level="state").stack(level="side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [166]:
df.unstack(level="state").stack(level="state")

side             left  right
number state                
one    Ohio         0      5
       Colorado     3      8
two    Ohio         1      6
       Colorado     4      9
three  Ohio         2      7
       Colorado     5     10

In [167]:
df.unstack(level="state").stack(level="state").unstack(level="number")

side     left           right          
number    one three two   one three two
state                                  
Ohio        0     2   1     5     7   6
Colorado    3     5   4     8    10   9

In [168]:
df.unstack(level="state").stack(level="state").unstack(level="number").stack(level="number")

side             left  right
state    number             
Ohio     one        0      5
         three      2      7
         two        1      6
Colorado one        3      8
         three      5     10
         two        4      9

### 8.3.2 Pivoting “Long” to “Wide” Format

A common way to store multiple time series in databases and CSV files is what is sometimes called long or stacked format. In this format, individual values are represented by a single row in a table rather than multiple values per row.

Let's load some example data and do a small amount of time series wrangling and other data cleaning.

In [186]:
data1 = pd.read_csv("examples/macrodata.csv")
data1.shape

(203, 14)

In [187]:
list(data1.columns)

['year',
 'quarter',
 'realgdp',
 'realcons',
 'realinv',
 'realgovt',
 'realdpi',
 'cpi',
 'm1',
 'tbilrate',
 'unemp',
 'pop',
 'infl',
 'realint']

In [188]:
data2 = data1.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data2

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2
...,...,...,...,...,...
198,2008,3,13324.600,-3.16,6.0
199,2008,4,13141.920,-8.79,6.9
200,2009,1,12925.410,0.94,8.1
201,2009,2,12901.504,3.37,9.2


In [189]:
periods = pd.PeriodIndex(year=data2.pop("year"),
                         quarter=data2.pop("quarter"),
                         name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [193]:
data2.index

RangeIndex(start=0, stop=203, step=1)

In [194]:
data2.index = periods.to_timestamp("D")
data2.index

DatetimeIndex(['1959-01-01', '1959-04-01', '1959-07-01', '1959-10-01',
               '1960-01-01', '1960-04-01', '1960-07-01', '1960-10-01',
               '1961-01-01', '1961-04-01',
               ...
               '2007-04-01', '2007-07-01', '2007-10-01', '2008-01-01',
               '2008-04-01', '2008-07-01', '2008-10-01', '2009-01-01',
               '2009-04-01', '2009-07-01'],
              dtype='datetime64[ns]', name='date', length=203, freq='QS-OCT')

In [195]:
data2.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [196]:
data3 = data2.reindex(columns=["realgdp", "infl", "unemp"])
data3.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [197]:
data3.columns.name = "item"
data3.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [203]:
long_data = (data3.stack()
             .reset_index()
             .rename(columns={0: "value"}))

In [204]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


> In this so-called long format for multiple time series, each row in the table represents a single observation.

In [206]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


In [208]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                
item        infl   realgdp unemp
date                            
1959-01-01  0.00  2710.349   5.8
1959-04-01  2.34  2778.801   5.1
1959-07-01  2.74  2775.488   5.3
1959-10-01  0.27  2785.204   5.6
1960-01-01  2.31  2847.699   5.2

### 8.3.3. Pivoting “Wide” to “Long” Format

Reading assignment